In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import numpy as np
import xarray as xr
import dask.array as da

from polsarpro.speckle_filters import refined_lee
from dask.diagnostics import ProgressBar

In [ ]:
# Load data and make dataset
input_data = Path("/data/psp/test_files/Simul_PolSAR.npy")
img = np.load(input_data)

dims = ("y", "x")
T3_dict = dict(
    m11=xr.DataArray(img[..., 0, 0].real, dims=dims),
    m22=xr.DataArray(img[..., 1, 1].real, dims=dims),
    m33=xr.DataArray(img[..., 2, 2].real, dims=dims),
    m12=xr.DataArray(img[..., 0, 1], dims=dims),
    m13=xr.DataArray(img[..., 0, 2], dims=dims),
    m23=xr.DataArray(img[..., 1, 2], dims=dims),    
)
# T3 = xr.Dataset(T3_dict, attrs=dict(poltype="T3", description="...")).chunk(x=512, y=512)
T3 = xr.Dataset(T3_dict, attrs=dict(poltype="T3", description="...")).chunk(x=128, y=128)

In [ ]:
with ProgressBar():
    res = refined_lee(T3, window_size=7, num_looks=4).compute()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(res.m11, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)